In [1]:
import random
import csv

In [3]:
random.seed(0)  # Set the random seed so the results are reproducible

# Given data
tasks = {
    "ax-cpt": 16.78833333,
    "cuedTS": 10.12166667,
    "flanker": 6.446666667,
    "go-nogo": 9.471666667,
    "nback": 9.271666667,
    "span": 23.68,
    "spatial_cueing": 14.64666667,
    "spatialTS": 10.12166667,
    "stop_signal": 9.046666667,
    "stroop": 6.446666667,
    "visual_search": 11.08,
}

BATTERY_LIMIT = 60  # in minutes

In [4]:
seen_batteries = set()  # Keep track of generated batteries to avoid duplicates


def generate_batteries():
    all_tasks = list(tasks.keys())
    random.shuffle(all_tasks)
    task_order = all_tasks * 5

    # Insert the survey task randomly into the task_order
    survey_position = random.randint(0, len(task_order))
    task_order.insert(survey_position, "survey")

    batteries = []
    current_battery = []
    current_time = 0

    for task in task_order:
        if current_time + (tasks[task] if task in tasks else 30) <= BATTERY_LIMIT:
            current_battery.append(task)
            current_time += tasks[task] if task in tasks else 30
        else:
            batteries.append(current_battery)
            current_battery = [task]
            current_time = tasks[task] if task in tasks else 30

    if current_battery:  # Add the last battery if there are remaining tasks
        batteries.append(current_battery)

    # Convert batteries to a frozenset so they can be stored in a set
    battery_tuple = tuple(frozenset(battery) for battery in batteries)

    # Check if these batteries have been seen before
    if battery_tuple in seen_batteries:
        return []

    seen_batteries.add(battery_tuple)
    return batteries


def calculate_battery_time(battery):
    return sum(tasks[task] if task in tasks else 30 for task in battery)


all_sets = []

# Keep trying until 12 batteries are created for each set
for _ in range(4):
    batteries = []
    while len(batteries) != 12:
        batteries = generate_batteries()
    all_sets.append(batteries)

# After generating the batteries, write them to a CSV file
with open("batteries.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write the header
    writer.writerow(["Set Number", "Battery Number", "Tasks", "Estimated Time (min)"])

    # Write the batteries for each set
    for set_num, battery_set in enumerate(all_sets, start=1):
        for i, battery in enumerate(battery_set, start=1):
            time_estimate = calculate_battery_time(battery)
            writer.writerow(
                [f"Set {set_num}", f"Battery {i}", ", ".join(battery), time_estimate]
            )
